In [4]:
print("Lets go")

Lets go


In [158]:
!pip -q install langchain_huggingface

In [5]:
import os
import warnings
from tqdm import tqdm
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.llms import CTransformers
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from pinecone import Pinecone, ServerlessSpec


/Users/soham/Documents/GitHub/MediBuddy./medienv/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
# Extract Data from PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader) 
    documents = loader.load()
    return documents


In [7]:
extracted_data=load_pdf('/Users/soham/Documents/GitHub/MediBuddy./Data')

In [ ]:
extracted_data

In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [11]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

/Users/soham/Documents/GitHub/MediBuddy./medienv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
api_key = os.getenv("PINECONE_API_KEY")
index_name = "medical-chatbot-implementation-extended"
pc = Pinecone(api_key=api_key)

In [167]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [16]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
print(api_key)


cfdae7d3-19c1-4ac1-bb8a-9275847ceae2


In [17]:
index_name = "medibuddy"
pc = Pinecone(api_key=api_key)

In [18]:
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [19]:

prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [20]:
llm = CTransformers(model = "/Users/soham/Documents/GitHub/MediBuddy./Llama_downloaded_model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [21]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x137258850>, model='/Users/soham/Documents/GitHub/MediBuddy./Llama_downloaded_model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

In [22]:
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'page': 130.0, 'source': '/Users/soham/Documents/GitHub/MediBuddy./Data/Medical_book.pdf'}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(metadata={'page': 129.0, 'source': '/Users/soham/Documents/GitHub/MediBuddy./Data/Medical_book.pdf'}, page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens 

In [23]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x12447bca0>)

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x12447bca0>)

In [32]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [34]:
while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() == 'q':
        print("Quitting the program.")
        break
    result = qa({"query": user_input})
    print("Response:", result["result"])

Response: Migraine is a type of headache disorder characterized by recurring attacks of moderate to severe head pain, often accompanied by other symptoms such as nausea, vomiting, and sensitivity to light and sound. The exact cause of migraines is not fully understood, but it is believed that changes in blood flow and neurotransmitter levels in the brain may play a role. Migraines can be triggered by a variety of factors, including hormonal changes, stress, certain foods or environmental triggers, and can significantly impact an individual's quality of life.
